[View in Colaboratory](https://colab.research.google.com/github/adityasingh1993/100DayMLChallenge/blob/master/CustomWordVec.ipynb)

In [0]:
import tensorflow as tf 
import numpy as np 
import re
from collections import Counter
import sys
import math
from random import randint
import pickle
import os
from __future__ import absolute_import, division, print_function
import codecs
import glob
import re
import multiprocessing
import nltk

In [0]:
wordVecDimensions = 100
batchSize = 128
numNegativeSample = 64
windowSize = 5
numIterations = 10000

In [0]:

book_filenames = sorted(glob.glob("data/*.txt"))
print("Found books:")
book_filenames

Found books:


['data/got1.txt',
 'data/got2.txt',
 'data/got3.txt',
 'data/got4.txt',
 'data/got5.txt',
 'data/scotus.txt']

In [0]:
corpus_raw=u""
for book_filename in book_filenames:
    print("reading_books..".format(book_filename))
    with codecs.open(book_filename,"r","utf-8") as book_file:
        corpus_raw+=book_file.read()
    print("length of corpus",len(corpus_raw))

reading_books..
length of corpus 1770659
reading_books..
length of corpus 4071041
reading_books..
length of corpus 6391405
reading_books..
length of corpus 8107945
reading_books..
length of corpus 9719485
reading_books..
length of corpus 22420105


In [0]:
def processDataset(corpus):
    finalDict = Counter(corpus.split())
    return  finalDict

In [0]:
def createTrainingMatrices(dictionary, corpus):
    allUniqueWords = list(dictionary)
    allWords = corpus.split()
    numTotalWords = len(allWords)
    xTrain=[]
    yTrain=[]
    for i in range(1000):
        if i % 10 == 0:
            print ("Finished %d/%d total words" % (i, numTotalWords))
        wordsAfter = allWords[i + 1:i + windowSize + 1]
        wordsBefore = allWords[max(0, i - windowSize):i]
        wordsAdded = wordsAfter + wordsBefore
        for word in wordsAdded:
            xTrain.append(allUniqueWords.index(allWords[i]))
            #xTrain.append(next(iter(allUniqueWords([allWords[i]]))))
            yTrain.append(allUniqueWords.index(word))
    return xTrain, yTrain

In [0]:
def getTrainingBatch():
    num = randint(0,numTrainingExamples - batchSize - 1)
    arr = xTrain[num:num + batchSize]
    labels = yTrain[num:num + batchSize]
   # labels1=labels[:,np.newaxis]
    labels1=np.array(labels)[:,np.newaxis]
   # print(labels1)
    return arr,np.array(labels1) 
    #return arr,np.array(labels).mean(axis=0)

In [0]:
continueWord2Vec = True


In [0]:
import dill
if (os.path.isfile('Word2VecXTrain.npy') and os.path.isfile('Word2VecYTrain.npy') and os.path.isfile('wordList.txt')):
    xTrain = np.load('Word2VecXTrain.npy')
    yTrain = np.load('Word2VecYTrain.npy')
    print('Finished loading training matrices')
    continueWord2Vec = False
    with open("wordList.txt", "rb") as fp:
        wordList = pickle.load(fp)
    print("Finished loading word list")
else:
    fullCorpus=corpus_raw
    datasetDictionary = processDataset(corpus_raw)
    print('Finished parsing and cleaning dataset')
    wordList = datasetDictionary.keys()
    xTrain, yTrain  = createTrainingMatrices(datasetDictionary, fullCorpus)
    print('Finished creating training matrices')
    np.save('Word2VecXTrain.npy', xTrain)
    np.save('Word2VecYTrain.npy', yTrain)
   
   
   

Finished parsing and cleaning dataset
Finished 0/4069446 total words
Finished 10/4069446 total words
Finished 20/4069446 total words
Finished 30/4069446 total words
Finished 40/4069446 total words
Finished 50/4069446 total words
Finished 60/4069446 total words
Finished 70/4069446 total words
Finished 80/4069446 total words
Finished 90/4069446 total words
Finished 100/4069446 total words
Finished 110/4069446 total words
Finished 120/4069446 total words
Finished 130/4069446 total words
Finished 140/4069446 total words
Finished 150/4069446 total words
Finished 160/4069446 total words
Finished 170/4069446 total words
Finished 180/4069446 total words
Finished 190/4069446 total words
Finished 200/4069446 total words
Finished 210/4069446 total words
Finished 220/4069446 total words
Finished 230/4069446 total words
Finished 240/4069446 total words
Finished 250/4069446 total words
Finished 260/4069446 total words
Finished 270/4069446 total words
Finished 280/4069446 total words
Finished 290/406

In [0]:
list1=list(wordList)
with open("wordList.txt",'wb') as f:
    dill.dump(list1, f)

In [0]:
print('Finished creating training matrices')
np.save('Word2VecXTrain.npy', xTrain)
np.save('Word2VecYTrain.npy', yTrain)
   
with open("wordList.txt", "wb") as fp: 
    pickle.dump(wordList, fp)

Finished creating training matrices


TypeError: can't pickle dict_keys objects

In [0]:
if (continueWord2Vec == False):
    sys.exit()

In [0]:
numTrainingExamples = len(xTrain)
vocabSize = len(wordList)

In [0]:
sess = tf.Session()
embeddingMatrix = tf.Variable(tf.random_uniform([vocabSize, wordVecDimensions], -1.0, 1.0))
nceWeights = tf.Variable(tf.truncated_normal([vocabSize, wordVecDimensions], stddev=1.0 / math.sqrt(wordVecDimensions)))
nceBiases = tf.Variable(tf.zeros([vocabSize]))

In [0]:
inputs = tf.placeholder(tf.int32, shape=[batchSize])
outputs = tf.placeholder(tf.int32, shape=[batchSize, 1])
embed = tf.nn.embedding_lookup(embeddingMatrix, inputs)


In [0]:
inputs.get_shape()

TensorShape([Dimension(128)])

In [0]:
loss = tf.reduce_mean(
    tf.nn.nce_loss(weights=nceWeights,
                   biases=nceBiases,
                   labels=outputs,
                   inputs=embed,
                   num_sampled=numNegativeSample,
                   num_classes=vocabSize))

In [0]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

In [0]:
print(xTrain)

[82731, 82731, 82731, 82731, 82731, 1279, 1279, 1279, 1279, 1279, 1279, 81269, 81269, 81269, 81269, 81269, 81269, 81269, 106270, 106270, 106270, 106270, 106270, 106270, 106270, 106270, 16991, 16991, 16991, 16991, 16991, 16991, 16991, 16991, 16991, 39217, 39217, 39217, 39217, 39217, 39217, 39217, 39217, 39217, 39217, 63623, 63623, 63623, 63623, 63623, 63623, 63623, 63623, 63623, 63623, 106270, 106270, 106270, 106270, 106270, 106270, 106270, 106270, 106270, 106270, 61065, 61065, 61065, 61065, 61065, 61065, 61065, 61065, 61065, 61065, 38365, 38365, 38365, 38365, 38365, 38365, 38365, 38365, 38365, 38365, 11793, 11793, 11793, 11793, 11793, 11793, 11793, 11793, 11793, 11793, 48912, 48912, 48912, 48912, 48912, 48912, 48912, 48912, 48912, 48912, 105328, 105328, 105328, 105328, 105328, 105328, 105328, 105328, 105328, 105328, 89929, 89929, 89929, 89929, 89929, 89929, 89929, 89929, 89929, 89929, 45101, 45101, 45101, 45101, 45101, 45101, 45101, 45101, 45101, 45101, 29724, 29724, 29724, 29724, 2972

In [0]:
sess.run(tf.global_variables_initializer())

In [0]:
for i in range(numIterations):
    trainInputs, trainLabels = getTrainingBatch()
    #print(trainInputs)
   # print(trainLabels)
    trainInputs1=np.array(trainInputs)
    trainInputs1=trainInputs1.reshape(128)
   
    trainLabels1=np.array(trainLabels)
    trainLabels1=trainLabels1.reshape(batchSize,1)
   # print(trainInputs.shape)
    _, curLoss = sess.run([optimizer,loss], feed_dict={inputs: trainInputs1, outputs: trainLabels1})
    if (i % 10 == 0):
        print ('Current loss is:', curLoss)
print('Saving the word embedding matrix')
embedMatrix = embeddingMatrix.eval(session=sess)
np.save('embeddingMatrix.npy', embedMatrix)

Current loss is: 129.09
Current loss is: 91.7516
Current loss is: 107.571
Current loss is: 90.5405
Current loss is: 89.6435
Current loss is: 147.336
Current loss is: 90.4104
Current loss is: 101.29
Current loss is: 98.3177
Current loss is: 107.622
Current loss is: 130.274
Current loss is: 228.186
Current loss is: 132.883
Current loss is: 170.61
Current loss is: 74.0162
Current loss is: 86.136
Current loss is: 120.215
Current loss is: 101.721
Current loss is: 138.184
Current loss is: 130.037
Current loss is: 92.1949
Current loss is: 98.4757
Current loss is: 112.445
Current loss is: 161.132
Current loss is: 113.799
Current loss is: 93.2137
Current loss is: 93.7572
Current loss is: 63.0454
Current loss is: 55.0113
Current loss is: 79.5681
Current loss is: 81.6838
Current loss is: 148.96
Current loss is: 123.63
Current loss is: 129.313
Current loss is: 149.819
Current loss is: 111.658
Current loss is: 87.8895
Current loss is: 111.947
Current loss is: 67.0943
Current loss is: 65.5967
Curren

Current loss is: 44.1253
Current loss is: 33.2032
Current loss is: 81.5667
Current loss is: 45.3443
Current loss is: 65.439
Current loss is: 25.4895
Current loss is: 52.7882
Current loss is: 117.956
Current loss is: 72.8314
Current loss is: 69.3187
Current loss is: 41.01
Current loss is: 50.2181
Current loss is: 44.6781
Current loss is: 34.3538
Current loss is: 51.8393
Current loss is: 54.4087
Current loss is: 47.8946
Current loss is: 82.1577
Current loss is: 79.24
Current loss is: 35.1372
Current loss is: 72.837
Current loss is: 43.2689
Current loss is: 69.9766
Current loss is: 78.7104
Current loss is: 46.9052
Current loss is: 64.1699
Current loss is: 44.5288
Current loss is: 75.1427
Current loss is: 37.3965
Current loss is: 36.3197
Current loss is: 72.3023
Current loss is: 18.5742
Current loss is: 38.4606
Current loss is: 60.7814
Current loss is: 38.7985
Current loss is: 36.1841
Current loss is: 63.2532
Current loss is: 36.2487
Current loss is: 28.0367
Current loss is: 28.1725
Curren

Current loss is: 54.7103
Current loss is: 63.7408
Current loss is: 66.1704
Current loss is: 28.1746
Current loss is: 25.447
Current loss is: 37.5848
Current loss is: 37.256
Current loss is: 55.7079
Current loss is: 72.7641
Current loss is: 46.8267
Current loss is: 40.3388
Current loss is: 48.4225
Current loss is: 42.6735
Current loss is: 27.1217
Current loss is: 43.1027
Current loss is: 46.4294
Current loss is: 0.199814
Current loss is: 37.8742
Current loss is: 54.5266
Current loss is: 19.9728
Current loss is: 27.7278
Current loss is: 49.5818
Current loss is: 36.2
Current loss is: 35.5457
Current loss is: 37.0498
Current loss is: 35.9634
Current loss is: 10.0884
Current loss is: 36.3532
Current loss is: 0.0228734
Current loss is: 10.287
Current loss is: 26.5227
Current loss is: 9.73483
Current loss is: 38.799
Current loss is: 40.004
Current loss is: 36.1019
Current loss is: 9.90072
Current loss is: 81.6741
Current loss is: 44.5451
Current loss is: 42.6005
Current loss is: 38.6179
Curre

Current loss is: 19.2206
Current loss is: 19.2867
Current loss is: 28.7504
Current loss is: 11.3246
Current loss is: 1.72439
Saving the word embedding matrix
